# Advanced Querying Mongo

Importing libraries and setting up connection

In [24]:
%pip install folium

Note: you may need to restart the kernel to use updated packages.


In [25]:
from pymongo import MongoClient

In [26]:
str_conn='mongodb://localhost:27017'
cursor=MongoClient(str_conn)
cursor

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [27]:
cliente=MongoClient()
db=cliente.companies
coleccion=db.coleccion

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [28]:
query={'name': 'Babelgum'}
list(coleccion.find(query))

[{'_id': ObjectId('52cdef7c4bab8bd675297da0'),
  'name': 'Babelgum',
  'permalink': 'babelgum',
  'crunchbase_url': 'http://www.crunchbase.com/company/babelgum',
  'homepage_url': 'http://babelgum.com',
  'blog_url': 'http://babelgum.com/blog',
  'blog_feed_url': 'http://feeds.feedburner.com/Babelgum',
  'twitter_username': 'Babelgum',
  'category_code': 'games_video',
  'number_of_employees': None,
  'founded_year': 2007,
  'founded_month': 3,
  'founded_day': 1,
  'deadpooled_year': 2013,
  'deadpooled_month': 1,
  'deadpooled_day': 9,
  'deadpooled_url': '',
  'tag_list': 'iptv, web2ireland',
  'alias_list': '',
  'email_address': 'info@babelgum.com',
  'phone_number': '',
  'description': '',
  'created_at': 'Sat Jun 09 08:15:21 UTC 2007',
  'updated_at': 'Wed Oct 16 06:30:25 UTC 2013',
  'overview': '<p>Babelgum is an integrated web and mobile video content platform, free for users and supported by advertising, available on-demand to a global audience.</p>\n\n<p>On March 20th 2009

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [29]:
query = {'number_of_employees': {'$gte': 5000}}

seleccion = {'_id': 0, 'name': 1, 'number_of_employees': 1}  #Excluir el id_0 y empezar a incluir los campos (1)

resultado = coleccion.find(query, seleccion).sort('number_of_employees', -1).limit(20)

list(resultado)

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000},
 {'name': 'Sony', 'number_of_employees': 180500},
 {'name': 'LG', 'number_of_employees': 177000},
 {'name': 'Ford', 'number_of_employees': 171000},
 {'name': 'Boeing', 'number_of_employees': 160000},
 {'name': 'Digital Equipment Corporation', 'number_of_employees': 140000},
 {'name': 'Nokia', 'number_of_employees': 125000},
 {'name': 'MItsubishi Electric', 'number_of_employees': 107000}

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [30]:
query={'$and': [{'founded_year':{'$gte':2000}},
                 {'founded_year':{'$lte':2005}}]}
       
seleccion={'_id':0, 'name':1, 'founded_year':1}

resultado = coleccion.find(query, seleccion).sort('founded_year', -1).limit(5)

list(resultado)
               

[{'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Jingle Networks', 'founded_year': 2005},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Wetpaint', 'founded_year': 2005}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [31]:
query={'$and': [{'ipo.valuation_amount': {'$gt':100000000}},
              {'founded_year':{'$lt':2010}}]}
               
seleccion={'_id':0,'name':1, 'ipo.valuation_amount':1}

resultado=coleccion.find(query,seleccion).sort('ipo.valuation_amount', -1).limit(5)
list(resultado)

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000.0}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000.0}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000.0}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000.0}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000.0}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [32]:
query={'$and':[{'number_of_employees':{'$lt': 1000}},
               {'founded_year':{'$gt':2005}}]}

seleccion={'_id':0, 'number_of_employees':1, 'founded_year':1}
resultado=coleccion.find(query,seleccion).sort('number_of_employees',-1).limit(10)
list(resultado)

[{'number_of_employees': 900, 'founded_year': 2007},
 {'number_of_employees': 900, 'founded_year': 2006},
 {'number_of_employees': 666, 'founded_year': 2011},
 {'number_of_employees': 650, 'founded_year': 2006},
 {'number_of_employees': 650, 'founded_year': 2006},
 {'number_of_employees': 600, 'founded_year': 2007},
 {'number_of_employees': 600, 'founded_year': 2009},
 {'number_of_employees': 550, 'founded_year': 2008},
 {'number_of_employees': 550, 'founded_year': 2007},
 {'number_of_employees': 540, 'founded_year': 2008}]

### 6. All the companies that don't include the `partners` field.

In [33]:
query={'providerships':{'$exists':'FALSE'}}
seleccion={'_id':0, 'name':1}
resultado=coleccion.find(query,seleccion).limit(5)
list(resultado)

[{'name': 'Wetpaint'},
 {'name': 'AdventNet'},
 {'name': 'Zoho'},
 {'name': 'Digg'},
 {'name': 'Facebook'}]

### 7. All the companies that have a null type of value on the `category_code` field.

In [34]:
query={'category_code':None}
seleccion={'_id':0, 'name':1}
resultado=coleccion.find(query,seleccion).limit(5)
list(resultado)


[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [35]:
query={'$and': [{'number_of_employees': {'$gte':100}},
              {'number_of_employees':{'$lte':1000}}]}

seleccion={'_id':0, 'name':1, 'number_of_employees':1}

resultado=coleccion.find(query,seleccion).limit(5)
list(resultado)

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110}]

### 9. Order all the companies by their IPO price in a descending order.

In [36]:
query={}
seleccion={'_id':0, 'name':1, 'ipo.valuation_amount':1}
resultado=coleccion.find(query,seleccion).sort('ipo.valuation_amount', -1).limit(10)
list(resultado)

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000.0}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000.0}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000.0}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000.0}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000.0}},
 {'name': 'Tencent', 'ipo': {'valuation_amount': 11000000000.0}},
 {'name': 'Western Digital', 'ipo': {'valuation_amount': 9430000000.0}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000.0}},
 {'name': 'BMC Software', 'ipo': {'valuation_amount': 6000000000.0}},
 {'name': 'Rackspace', 'ipo': {'valuation_amount': 5440000000.0}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [37]:
query={}
seleccion={'_id':0, 'name':1, 'number_of_employees':1}
resultado=coleccion.find(query,seleccion).sort('number_of_employees',-1).limit(10)
list(resultado)

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [38]:
query={'$and':[{'founded_month': {'$gte':6}}, 
               {'founded_month': {'$lte':12}}]}

seleccion={'_id':0, 'name':1,'founded_month':1}

resultado=coleccion.find(query,seleccion).sort('founded_month', -1).limit(1000)

list(resultado)
       

[{'name': 'Web CEO', 'founded_month': 12},
 {'name': 'Cleantech Republic', 'founded_month': 12},
 {'name': 'Zhift', 'founded_month': 12},
 {'name': 't5m', 'founded_month': 12},
 {'name': 'GamerVixens', 'founded_month': 12},
 {'name': 'Logomyway', 'founded_month': 12},
 {'name': 'NetroStar', 'founded_month': 12},
 {'name': 'GoodGuide', 'founded_month': 12},
 {'name': 'Muckety', 'founded_month': 12},
 {'name': 'OnTheRoad', 'founded_month': 12},
 {'name': 'Macronimous Web Solutions', 'founded_month': 12},
 {'name': 'Revealrealestate', 'founded_month': 12},
 {'name': 'Newstree', 'founded_month': 12},
 {'name': 'Delhitoagra', 'founded_month': 12},
 {'name': 'Siruna', 'founded_month': 12},
 {'name': 'BlueCat Networks', 'founded_month': 12},
 {'name': 'HauteLook', 'founded_month': 12},
 {'name': 'PhoneFavs', 'founded_month': 12},
 {'name': 'FlowPlay', 'founded_month': 12},
 {'name': 'Styky', 'founded_month': 12},
 {'name': 'OBRA Incorporated', 'founded_month': 12},
 {'name': 'LionDrive', 'fou

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [39]:
query={'$and': [{'founded_year':{'$lt':2000}},
               {'acquisition.price_amount': {'$gt':10000000}}]}
seleccion={'_id':0, 'name':1, 'founded_year':1, 'acquisition.price_amount':1}
resultado=coleccion.find(query,seleccion).sort('acquisition.price_amount',-1).limit(10)
list(resultado)

[{'name': 'BEA Systems',
  'founded_year': 1995,
  'acquisition': {'price_amount': 8500000000.0}},
 {'name': 'Navteq',
  'founded_year': 1985,
  'acquisition': {'price_amount': 8100000000.0}},
 {'name': 'Sun Microsystems',
  'founded_year': 1982,
  'acquisition': {'price_amount': 7400000000.0}},
 {'name': 'Pixar',
  'founded_year': 1986,
  'acquisition': {'price_amount': 7400000000.0}},
 {'name': 'LSI',
  'founded_year': 1980,
  'acquisition': {'price_amount': 6600000000.0}},
 {'name': 'National Semiconductor',
  'founded_year': 1959,
  'acquisition': {'price_amount': 6500000000.0}},
 {'name': 'aQuantive',
  'founded_year': 1997,
  'acquisition': {'price_amount': 6400000000.0}},
 {'name': 'Siebel Systems',
  'founded_year': 1993,
  'acquisition': {'price_amount': 5850000000.0}},
 {'name': 'Sybase',
  'founded_year': 1984,
  'acquisition': {'price_amount': 5800000000.0}},
 {'name': 'Affiliated Computer Services',
  'founded_year': 1988,
  'acquisition': {'price_amount': 5750000000.0}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [40]:
query={'founded_year':{'$gt':2010}}
seleccion={'_id':0, 'name':1, 'acquisition.price_amount':-1}
resultado=coleccion.find(query,seleccion).sort('acquisition.price_amount',-1).limit(10)
list(resultado)


[{'name': 'RazorGator'},
 {'name': 'Advaliant'},
 {'name': 'Fixya'},
 {'name': 'Widgetbox'},
 {'name': 'Wamba'},
 {'name': 'headr'},
 {'name': 'CircleUp'},
 {'name': 'Mokitown'},
 {'name': 'PeekYou'},
 {'name': 'Pinger'}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [41]:
query={}
seleccion={'_id':0, 'name':1, 'founded_year':1}
resultado=coleccion.find(query,seleccion).sort('founded_year',-1).limit(10)
list(resultado)

[{'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Gimigo', 'founded_year': 2013},
 {'name': 'Clowdy', 'founded_year': 2013},
 {'name': 'Fixya', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'SEOGroup', 'founded_year': 2013},
 {'name': 'Pikk', 'founded_year': 2013},
 {'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [42]:
query={'$and':[{'founded_day':{'$gte':1}},
              {'founded_day':{'$lte':7}}]}
seleccion={'_id':0,'name':1, 'founded_day':1, 'acquisition.price_amount':1}
resultado=coleccion.find(query,seleccion).sort('acquisition.price_amount',-1).limit(10)
list(resultado)

[{'name': 'Netscape',
  'founded_day': 4,
  'acquisition': {'price_amount': 4200000000.0}},
 {'name': 'PayPal',
  'founded_day': 1,
  'acquisition': {'price_amount': 1500000000}},
 {'name': 'Zappos',
  'founded_day': 1,
  'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba',
  'founded_day': 1,
  'acquisition': {'price_amount': 1000000000}},
 {'name': 'Postini',
  'founded_day': 2,
  'acquisition': {'price_amount': 625000000}},
 {'name': 'Danger',
  'founded_day': 1,
  'acquisition': {'price_amount': 500000000}},
 {'name': 'Clearwell Systems',
  'founded_day': 6,
  'acquisition': {'price_amount': 410000000}},
 {'name': 'PrimeSense',
  'founded_day': 1,
  'acquisition': {'price_amount': 345000000}},
 {'name': 'Amobee',
  'founded_day': 1,
  'acquisition': {'price_amount': 321000000}},
 {'name': 'BlueLithium',
  'founded_day': 1,
  'acquisition': {'price_amount': 300000000}}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [43]:
query = {"category_code": "web", 
         "number_of_employees": { "$gt": 4000 }}
seleccion = {"_id": 0, "name": 1, "number_of_employees": 1}
resultado = coleccion.find(query, seleccion).sort("number_of_employees", 1)
list(resultado)



[{'name': 'Expedia', 'number_of_employees': 4400},
 {'name': 'AOL', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'number_of_employees': 8657},
 {'name': 'Rakuten', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group', 'number_of_employees': 10000},
 {'name': 'Groupon', 'number_of_employees': 10000},
 {'name': 'Yahoo!', 'number_of_employees': 13600},
 {'name': 'eBay', 'number_of_employees': 15000},
 {'name': 'Experian', 'number_of_employees': 15500}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [44]:
query = {"acquisition.price_amount": { "$gt": 10000000 },
         "acquisition.price_currency_code": "EUR"}
seleccion = {"_id": 0, "name": 1, "acquisition.price_amount": 1,"acquisition.price_currency_code": 1}
resultado = coleccion.find(query,seleccion)
list(resultado)


[{'name': 'ZYB',
  'acquisition': {'price_amount': 31500000, 'price_currency_code': 'EUR'}},
 {'name': 'Apertio',
  'acquisition': {'price_amount': 140000000, 'price_currency_code': 'EUR'}},
 {'name': 'Greenfield Online',
  'acquisition': {'price_amount': 40000000, 'price_currency_code': 'EUR'}},
 {'name': 'Webedia',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'Wayfinder',
  'acquisition': {'price_amount': 24000000, 'price_currency_code': 'EUR'}},
 {'name': 'Tuenti Technologies',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'BioMed Central',
  'acquisition': {'price_amount': 43400000, 'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [50]:
query = {'acquisition.acquired_month':{'$lte':3}}
select = {'_id':0, 'name':1,'acquisition.acquired_month':1 }
list(coleccion.find(query,select).limit(10))

[{'name': 'Kyte', 'acquisition': {'acquired_month': 1}},
 {'name': 'NetRatings', 'acquisition': {'acquired_month': 2}},
 {'name': 'blogTV', 'acquisition': {'acquired_month': 3}},
 {'name': 'Livestream', 'acquisition': {'acquired_month': 1}},
 {'name': 'iContact', 'acquisition': {'acquired_month': 2}},
 {'name': 'Coghead', 'acquisition': {'acquired_month': 2}},
 {'name': 'Dailymotion', 'acquisition': {'acquired_month': 2}},
 {'name': 'Netvibes', 'acquisition': {'acquired_month': 2}},
 {'name': 'Flickr', 'acquisition': {'acquired_month': 3}},
 {'name': 'BabyCenter', 'acquisition': {'acquired_month': 3}}]

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [22]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.